In [1]:
# importing required libraries 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from datetime import date
from datetime import datetime
import pandas as pd
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# installing the chrome driver & URL to be scrapped 
driver= webdriver.Chrome(r"C:\Users\arkay\Downloads\chromedriver_win32 (1)\chromedriver.exe")
url='https://in.indeed.com/'
driver.get(url)
driver.maximize_window()
time.sleep(5)

In [3]:
# Using selenium automation to reach the desired page through code 
driver.find_element_by_xpath("//*[@id='text-input-what']").send_keys('Data Analyst')
driver.find_element_by_xpath("//*[@id='text-input-where']").send_keys('bangalore')
driver.find_element_by_xpath("//*[@id='whatWhereFormId']/div[3]/button").click()

In [5]:
# Let's write the codes
driver.implicitly_wait(1)
titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []
descriptions=[]
Time=[]
Date=[]
days_posted=[]

In [8]:
# lets scrap first 20 pages
for i in range(0,20):
    
    job_posts= driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')
    
    for job in job_posts:
       
    #.  not all companies have review
        try:
            review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            location = job.find_element_by_xpath('.//span[contains(@class,"location")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)
        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
        companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)
        
        today=date.today()
        Date.append(today.strftime("%d/%m/%Y"))
        now=datetime.now()
        Time.append(now.strftime("%H:%M:%S"))
        
        try:
            dp=job.find_element_by_class_name("date ").text
        except:
            dp='None'
            days_posted.append(dp)
    
    try:
        next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
        next_page.click()

    except:
        next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
        next_page.click()
    #except:
        #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
        #next_page.click()
        

In [9]:
descriptions=[]
for link in links:
    
    driver.get(link)
    try:
        jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    except:
        jd='None'
    descriptions.append(jd)

In [10]:
df_Indeed=pd.DataFrame()
df_Indeed['Title']=titles
df_Indeed['Company']=companies
df_Indeed['Location']=locations
df_Indeed['Link']=links
df_Indeed['Review']=reviews
df_Indeed['Salary']=salaries
df_Indeed['Description']=descriptions
df_Indeed['Crawl Time']=Time
df_Indeed['Crawl Date']=Date

In [11]:
df_Indeed.head()

,Title,Company,Location,Link,Review,Salary,Description,Crawl Time,Crawl Date
0,"Analyst, Sales Operations",Dell Technologies,None,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...,4.0,None,Sales Operations Analyst\nBangalore\nCompany D...,20:12:21,02/01/2021
1,MIS/Data Analyst - ElasticRun,ElasticRun,"Bengaluru, Karnataka",https://in.indeed.com/rc/clk?jk=995c69d329a9fd...,1.5,None,Job Description\nRoles and Responsibilities\nL...,20:12:23,02/01/2021
2,Product Analyst,Mitron TV Pvt Ltd,"Bengaluru, Karnataka",https://in.indeed.com/company/Mitron-TV-Pvt-Lt...,None,"₹10,00,000 - ₹20,00,000 a year",Mitron - A series funded startup in short vide...,20:12:24,02/01/2021
3,Data Quality Analyst,Introlligent,"Bengaluru, Karnataka",https://in.indeed.com/company/Introlligent/job...,4.0,None,Job Expectations:\nExperience with Data Qualit...,20:12:26,02/01/2021
4,Data Analyst - Bill Validation,Proziod Analytics Pvt Ltd,"Koramangala, Bengaluru, Karnataka",https://in.indeed.com/company/Proziod-Analytic...,3.5,"₹3,00,000 a year",Job Description for the post of Data Analyst-B...,20:12:26,02/01/2021


In [12]:
df_Indeed.to_csv('df_Indeednewassignment.csv')